In [81]:
import pandas as pd

# Load the dataset to see the first few rows and understand its structure
train_data = pd.read_csv('/Users/shalu/Downloads/traindata.csv')
test_data = pd.read_csv('/Users/shalu/Downloads/testdata.csv')

# Display the first few rows of each dataset to understand their structure
train_data.head(), test_data.head()


/var/folders/jj/dsyly3wj01gdyv63g99v3tyw0000gn/T/ipykernel_67827/2396775270.py:4: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/Users/shalu/Downloads/traindata.csv')


(                                         NAME  Total Occupied housing units  \
 0              Benítez comunidad, Puerto Rico                         414.0   
 1        Mariano Colón comunidad, Puerto Rico                         679.0   
 2  Luis Lloréns Torres comunidad, Puerto Rico                         770.0   
 3             Boquerón comunidad, Puerto Rico                         421.0   
 4             El Mangó comunidad, Puerto Rico                         595.0   
 
    Median income of occupied housing units         Geography  state  place  \
 0                                  10903.0  1600000US7207496     72   7496   
 1                                  16649.0  1600000US7251055     72  51055   
 2                                  12308.0  1600000US7247615     72  47615   
 3                                  13385.0  1600000US7208099     72   8099   
 4                                  24803.0  1600000US7225556     72  25556   
 
    Year  Real estate taxes  Property Valu

In [82]:
# Standardize column names in the test data based on an educated guess of their correspondences with the training data


# Display the updated first few rows of test data
test_data.head()
# Handling Missing Values:
# Replace -666666666.0 in 'Real estate taxes' with NaN for consistent missing value handling
train_data['Real estate taxes'].replace(-666666666.0, pd.NA, inplace=True)
test_data['Real estate taxes'].replace(-666666666.0, pd.NA, inplace=True)

# Fill missing values with median for continuous variables in both datasets
for column in train_data.select_dtypes(include=['float64', 'int64']).columns:
    median_value_train = train_data[column].median()
    train_data[column].fillna(median_value_train, inplace=True)
    
    if column in test_data.columns:
        median_value_test = test_data[column].median()
        test_data[column].fillna(median_value_test, inplace=True)

# Check for any other missing values in categorical data
missing_values_train = train_data.isnull().sum()
missing_values_test = test_data.isnull().sum()

# Ensure all numeric data are of type float or int, and convert if necessary
for column in train_data.columns:
    if train_data[column].dtype == 'object':
        try:
            train_data[column] = pd.to_numeric(train_data[column])
        except ValueError:
            pass  # If conversion fails, it means the column is categorical

for column in test_data.columns:
    if test_data[column].dtype == 'object':
        try:
            test_data[column] = pd.to_numeric(test_data[column])
        except ValueError:
            pass  # If conversion fails, it means the column is categorical

# Display the status of missing values and data types after conversion
missing_values_train, missing_values_test, train_data.dtypes, test_data.dtypes


(NAME                                            0
 Total Occupied housing units                    0
 Median income of occupied housing units         0
 Geography                                       0
 state                                           0
 place                                           0
 Year                                            0
 Real estate taxes                           40068
 Property Value                                  0
 Yearly housing costs                            0
 Households below poverty line                   0
 Unemployed population                           0
 Mean till 40k                                   0
 Mean 40k-100k                                   0
 Above mean 100 K                                0
 Family Income by single earner household        0
 Income Spent1                                   0
 Income Spent                                    0
 dtype: int64,
 NAME                                         0
 Total Occupied hou

In [83]:
import numpy as np
# Prepare the data for prediction using only 'Property Value' as the predictor
data_for_regression_single = train_data[train_data['Real estate taxes'].notna()]

# Features and target using only 'Property Value'
X_single = data_for_regression_single[['Property Value']].values
y_single = data_for_regression_single['Real estate taxes'].values

# Add a column of ones to X_single to account for the intercept
X_single = np.concatenate([np.ones((X_single.shape[0], 1)), X_single], axis=1)

# Calculate the coefficients using the Normal Equation for single predictor
beta_single = np.linalg.inv(X_single.T.dot(X_single)).dot(X_single.T).dot(y_single)

# Prepare the data for prediction where 'Real estate taxes' is missing in the training set
missing_taxes_train_single = train_data[train_data['Real estate taxes'].isna()]
X_missing_train_single = missing_taxes_train_single[['Property Value']].values
X_missing_train_single = np.concatenate([np.ones((X_missing_train_single.shape[0], 1)), X_missing_train_single], axis=1)

# Predict the missing 'Real estate taxes' using the single predictor model
predicted_taxes_train_single = X_missing_train_single.dot(beta_single)

# Impute these values back into the original training dataset
train_data.loc[train_data['Real estate taxes'].isna(), 'Real estate taxes'] = predicted_taxes_train_single

# Prepare and predict for the test set
missing_taxes_test_single = test_data[test_data['Real estate taxes'].isna()]
X_missing_test_single = missing_taxes_test_single[['Property Value']].values
X_missing_test_single = np.concatenate([np.ones((X_missing_test_single.shape[0], 1)), X_missing_test_single], axis=1)
predicted_taxes_test_single = X_missing_test_single.dot(beta_single)
test_data.loc[test_data['Real estate taxes'].isna(), 'Real estate taxes'] = predicted_taxes_test_single

# Check if all missing values are filled
missing_values_train_single = train_data['Real estate taxes'].isna().sum()
missing_values_test_single = test_data['Real estate taxes'].isna().sum()

missing_values_train_single, missing_values_test_single, beta_single


(0, 0, array([-2.78373739e+06,  1.50855928e-01]))

In [84]:
train_data.drop(['NAME', 'state', 'place'], axis=1, inplace=True)
test_data.drop(['NAME', 'state', 'place'], axis=1, inplace=True)

# Display the updated datasets to confirm the columns have been dropped
train_data.head(), test_data.head()

(   Total Occupied housing units  Median income of occupied housing units  \
 0                         414.0                                  10903.0   
 1                         679.0                                  16649.0   
 2                         770.0                                  12308.0   
 3                         421.0                                  13385.0   
 4                         595.0                                  24803.0   
 
           Geography  Year  Real estate taxes  Property Value  \
 0  1600000US7207496  2010      -2.770688e+06         86500.0   
 1  1600000US7251055  2010      -2.773147e+06         70200.0   
 2  1600000US7247615  2010       4.056000e+03         96600.0   
 3  1600000US7208099  2010       1.990000e+02         95900.0   
 4  1600000US7225556  2010       1.990000e+02        171600.0   
 
    Yearly housing costs  Households below poverty line  Unemployed population  \
 0                2028.0                          245.0       

In [85]:
train_data.drop(['Income Spent1'], axis=1, inplace=True)
test_data.drop(['Income Spent1'], axis=1, inplace=True)

In [86]:
# Step 1: Data Type Conversion
# Convert all numeric columns to the appropriate types if they are not already set
numeric_columns = train_data.select_dtypes(include=['object']).columns.tolist()
for col in numeric_columns:
    try:
        train_data[col] = pd.to_numeric(train_data[col])
        test_data[col] = pd.to_numeric(test_data[col])
    except ValueError:
        print(f"Skipping {col}, not a numeric column.")

# Check current data types to ensure conversion where possible

train_data['Income Spent'] = pd.to_numeric(train_data['Income Spent'], errors='coerce')

# Check data types again after conversion
train_data.dtypes,test_data.dtypes

Skipping Geography, not a numeric column.
Skipping Income Spent, not a numeric column.


(Total Occupied housing units                float64
 Median income of occupied housing units     float64
 Geography                                    object
 Year                                          int64
 Real estate taxes                           float64
 Property Value                              float64
 Yearly housing costs                        float64
 Households below poverty line               float64
 Unemployed population                       float64
 Mean till 40k                               float64
 Mean 40k-100k                               float64
 Above mean 100 K                            float64
 Family Income by single earner household    float64
 Income Spent                                float64
 dtype: object,
 Total Occupied housing units                  int64
 Median income of occupied housing units       int64
 Geography                                    object
 Year                                          int64
 Real estate taxes            

In [87]:
# Step 2: Outlier Detection and Treatment
def detect_and_treat_outliers(df, column):
    # Calculate Q1, Q3, and IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    # Determine the outlier cutoffs
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Cap and floor outliers
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])

# Apply outlier treatment to key columns
key_columns = ['Real estate taxes', 'Property Value', 'Yearly housing costs']
for col in key_columns:
    detect_and_treat_outliers(train_data, col)
    detect_and_treat_outliers(test_data, col)

# Check statistics after outlier treatment to confirm changes
{col: train_data[col].describe() for col in key_columns}, {col: test_data[col].describe() for col in key_columns}


({'Real estate taxes': count    358887.000000
  mean        977.886420
  std        1943.339383
  min       -2892.000000
  25%           4.200000
  50%         926.000000
  75%        1935.000000
  max        4831.200000
  Name: Real estate taxes, dtype: float64,
  'Property Value': count    358887.000000
  mean     127839.194855
  std      127295.623069
  min     -105400.000000
  25%       72500.000000
  50%      114100.000000
  75%      191100.000000
  max      369000.000000
  Name: Property Value, dtype: float64,
  'Yearly housing costs': count    358887.000000
  mean       4890.400405
  std        5766.841638
  min      -11688.000000
  25%         192.000000
  50%        2184.000000
  75%        8112.000000
  max       19992.000000
  Name: Yearly housing costs, dtype: float64},
 {'Real estate taxes': count     681.000000
  mean      827.936123
  std       419.888176
  min      -146.500000
  25%       560.000000
  50%       727.000000
  75%      1031.000000
  max      1737.500000
  

In [88]:
train_data.drop(['Geography'], axis=1, inplace=True)
test_data.drop(['Geography'], axis=1, inplace=True)

In [89]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# List of columns to scale
columns_to_scale = ['Total Occupied housing units', 'Property Value', 'Yearly housing costs',
                    'Households below poverty line', 'Unemployed population', 'Mean till 40k', 
                    'Mean 40k-100k', 'Above mean 100 K', 'Family Income by single earner household']

# Apply scaling to both train and test datasets
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Display the scaled features to confirm scaling
train_data[columns_to_scale].head(), test_data[columns_to_scale].head()


(   Total Occupied housing units  Property Value  Yearly housing costs  \
 0                      0.000127        0.404511              0.432955   
 1                      0.000209        0.370152              0.451894   
 2                      0.000237        0.425801              0.446970   
 3                      0.000130        0.424325              0.427273   
 4                      0.000183        0.583895              0.479924   
 
    Households below poverty line  Unemployed population  Mean till 40k  \
 0                       0.000406               0.999493       0.000286   
 1                       0.000601               0.999493       0.000472   
 2                       0.000805               0.999493       0.000527   
 3                       0.000449               0.999493       0.000280   
 4                       0.000326               0.999493       0.000316   
 
    Mean 40k-100k  Above mean 100 K  Family Income by single earner household  
 0       0.000047     

In [107]:
import numpy as np
import pandas as pd

def calculate_mse(target):
    """ Calculate the mean squared error of the target variable """
    if len(target) == 0:
        return 0
    target_mean = target.mean()
    mse = ((target - target_mean) ** 2).mean()
    return mse

def best_split(data, min_samples_split=10, num_splits=10):
    if len(data) < min_samples_split:
        return None, None

    best_reduction = 0
    best_feature = None
    best_value = None
    current_mse = calculate_mse(data['Income Spent'])

    for feature in data.columns.drop('Income Spent'):
        # Use quantiles to determine split points rather than unique values
        split_points = np.quantile(data[feature].dropna(), np.linspace(0, 1, num_splits, endpoint=False)[1:])
        for value in split_points:
            left = data[data[feature] <= value]
            right = data[data[feature] > value]
            
            if len(left) < min_samples_split or len(right) < min_samples_split:
                continue

            left_mse = calculate_mse(left['Income Spent'])
            right_mse = calculate_mse(right['Income Spent'])
            weighted_mse = (len(left) / len(data)) * left_mse + (len(right) / len(data)) * right_mse
            
            mse_reduction = current_mse - weighted_mse
            if mse_reduction > best_reduction:
                best_reduction = mse_reduction
                best_feature = feature
                best_value = value

    return best_feature, best_value

class DecisionNode:
    """ Node in the decision tree """
    def __init__(self, feature=None, value=None, left=None, right=None, output=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.output = output

def build_tree(data, depth=0, max_depth=10):
    """ Recursively build the decision tree """
    if depth == max_depth or len(data) <= 1:
        node_mse = calculate_mse(data['Income Spent'])
        
        return DecisionNode(output=data['Income Spent'].mean())
    
    feature, value = best_split(data)
    if feature is None:
        return DecisionNode(output=data['Income Spent'].mean())
    
    left_data = data[data[feature] <= value]
    right_data = data[data[feature] > value]

    left_node = build_tree(left_data, depth+1, max_depth)
    right_node = build_tree(right_data, depth+1, max_depth)
    
    return DecisionNode(feature, value, left_node, right_node)

def predict(tree, instance):
    """ Predict target value for a single instance """
    if tree.output is not None:
        return tree.output
    if instance[tree.feature] <= tree.value:
        return predict(tree.left, instance)
    else:
        return predict(tree.right, instance)

def calculate_r2(tree, data):
    """ Calculate R² score for the decision tree """
    predictions = data.apply(lambda instance: predict(tree, instance), axis=1)
    target_mean = data['Income Spent'].mean()
    ss_tot = ((data['Income Spent'] - target_mean) ** 2).sum()
    ss_res = ((data['Income Spent'] - predictions) ** 2).sum()
    return 1 - ss_res / ss_tot

# Assuming train_data and test_data are your preprocessed datasets
try:
    tree = build_tree(train_data, max_depth=10)
    final_predictions = test_data.apply(lambda instance: predict(tree, instance), axis=1)
    r2_score = calculate_r2(tree, test_data)
    final_mse = calculate_mse(final_predictions - test_data['Income Spent'])
    print(f"R² Score: {r2_score}")
 
    print(f"Final Mean Squared Error: {final_mse}")
except Exception as e:
    print(f"Failed to build the tree or calculate metrics: {e}")


R² Score: 0.8165240357358197
Final Mean Squared Error: 4.9872820078805296


In [116]:
import numpy as np
import pandas as pd

# Assuming 'train_data' and 'test_data' are preloaded DataFrames
data = pd.concat([train_data, test_data])
features = data.drop('Income Spent', axis=1).values
target = data['Income Spent'].values

# Normalize features manually
features_mean = features.mean(axis=0)
features_std = features.std(axis=0)
features_normalized = (features - features_mean) / features_std

# Normalize target manually
target_mean = target.mean()
target_std = target.std()
target_normalized = (target - target_mean) / target_std

# Split data manually
split_ratio = 0.8
split_index = int(features_normalized.shape[0] * split_ratio)
X_train = features_normalized[:split_index]
X_test = features_normalized[split_index:]
y_train = target_normalized[:split_index]
y_train = y_train.reshape(-1, 1)  # Reshaping for matrix operations
y_test = target_normalized[split_index:]
y_test = y_test.reshape(-1, 1)  # Reshaping for matrix operations


In [117]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(2)  # Seed for reproducibility
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def forward_propagation(X, parameters):
    W1, b1, W2, b2 = parameters['W1'], parameters['b1'], parameters['W2'], parameters['b2']
    Z1 = np.dot(W1, X.T) + b1  # Shape of Z1 should be (n_h, m)
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = Z2  # Linear activation for output layer
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

def compute_cost(A2, Y):
    m = Y.shape[1]
    cost = (1./m) * np.sum((A2 - Y)**2)
    return cost

def backward_propagation(parameters, cache, X, Y):
    m = Y.shape[1]
    W1, W2 = parameters['W1'], parameters['W2']
    A1, A2 = cache['A1'], cache['A2']
    dZ2 = A2 - Y
    dW2 = (1./m) * np.dot(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(cache['Z1'])
    dW1 = (1./m) * np.dot(dZ1, X)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

def update_parameters(parameters, grads, learning_rate=0.01):
    W1, b1, W2, b2 = parameters['W1'], parameters['b1'], parameters['W2'], parameters['b2']
    dW1, db1, dW2, db2 = grads['dW1'], grads['db1'], grads['dW2'], grads['db2']
    parameters['W1'] -= learning_rate * dW1
    parameters['b1'] -= learning_rate * db1
    parameters['W2'] -= learning_rate * dW2
    parameters['b2'] -= learning_rate * db2
    return parameters


In [123]:
def mean_squared_error(y_true, y_pred):
    """Calculate the mean squared error between true and predicted values"""
    return np.mean((y_true - y_pred) ** 2)

def r_squared(y_true, y_pred):
    """Calculate the R² score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

# Function to predict using the neural network
def predict(parameters, X):
    A2, _ = forward_propagation(X, parameters)
    return A2.T  # Transpose to match the shape of Y

# Train the model
parameters = model(X_train, y_train, n_h=5, num_iterations=15000, print_cost=True)

# Make predictions on the training and test set
y_train_pred = predict(parameters, X_train)
y_test_pred = predict(parameters, X_test)

# Calculate and print the MSE and R² score for both training and test sets
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r_squared(y_train, y_train_pred)
test_r2 = r_squared(y_test, y_test_pred)


print(f"MSE: {test_mse}")

print(f"R² Score: {test_r2}")


Cost after iteration 0: 1.0212593481601704
Cost after iteration 1000: 0.86818023816323
Cost after iteration 2000: 0.8206650107963095
Cost after iteration 3000: 0.7760672304186192
Cost after iteration 4000: 0.7350318295916762
Cost after iteration 5000: 0.7031301294489316
Cost after iteration 6000: 0.675680532138604
Cost after iteration 7000: 0.642456480378803
Cost after iteration 8000: 0.5913155210101502
Cost after iteration 9000: 0.5117956787071953
Cost after iteration 10000: 0.40696970316697295
Cost after iteration 11000: 0.3026959621119501
Cost after iteration 12000: 0.23502296934529932
Cost after iteration 13000: 0.20588925938842007
Cost after iteration 14000: 0.1955398773318516
MSE: 0.11450592116780015
R² Score: 0.8735922263108942
